In [5]:
from langchain_openai import ChatOpenAI
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
store = {}

In [ ]:
def parentChatbotSummarize(filepath: str,session_id: str ,promptQuestion: str ):
    llm = ChatOpenAI(model = 'gpt-4o')

    loader = JSONLoader(file_path=filepath, jq_schema='.', text_content=False)
    docs=loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap= 200,add_start_index=True)
    splits = text_splitter.split_documents(docs)
    vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding= OpenAIEmbeddings())
    retriever = vectorstore.as_retriever(search_type= 'similarity')
    
    contextualize_q_system_prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed and otherwise return it as is."
    )

    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

    history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)
    system_prompt = (
        "You are a nurse updating a parent on the status of their child in the NICU."
        "Use the following retrieved context to provide relevant information to the parent. "
        "Speak in a conversational tone, and keep the summary concise."
        "Do not diagnose."
        "\n\n{context}"
    )

    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

    conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
    response = conversational_rag_chain.invoke(
        {"input": promptQuestion},
        config={
            "configurable": {"session_id": session_id}
        },  # constructs a key "abc123" in `store`.
    )

    return response['answer'] 

In [8]:
parentChatbotSummarize('Adolf_Data.json','TestTest','How hasmy child been?')

Document(metadata={'source': 'C:\\Users\\elfo\\Documents\\Capstone\\ChatBots_NICU_JSON_SUMMARY\\Adolf_Data.json', 'seq_num': 1, 'start_index': 0}, page_content='{"patient_id": "Adolf", "data": [{"timestamp": "2024-11-07 20:38:37", "vital_signs": {"heart_rate": 124, "respiratory_rate": 64, "body_temperature": 37.3}, "intervention": null, "pain_level": "Low"}, {"timestamp": "2024-11-07 20:43:37", "vital_signs": {"heart_rate": 138, "respiratory_rate": 75, "body_temperature": 37.1}, "intervention": null, "pain_level": "Low"}, {"timestamp": "2024-11-07 20:48:37", "vital_signs": {"heart_rate": 134, "respiratory_rate": 62, "body_temperature": 36.7}, "intervention": {"type": "feeding", "detail": "30 ml formula"}, "pain_level": "Low"}, {"timestamp": "2024-11-07 20:53:37", "vital_signs": {"heart_rate": 128, "respiratory_rate": 68, "body_temperature": 37.3}, "intervention": null, "pain_level": "Low"}, {"timestamp": "2024-11-07 20:58:37", "vital_signs": {"heart_rate": 102, "respiratory_rate": 94, 